# This is a learning project. It aims to have hands on experience in NLP

### Trying to build a model that predicts which Product do the customer complaints concern
### Will work with [Customer Complaint Database](https://www.kaggle.com/datasets/selener/consumer-complaint-database) from Kaggle

In [4]:
import pandas as pd
import numpy as np
import os
import json

In [62]:
SPARSITY_THRESHOLD = 0.5

def optimize(df: pd.DataFrame) -> pd.DataFrame:
    float_types = {np.finfo(np.float16).max: np.float16, np.finfo(np.float32).max: np.float32}
    int_types = {np.iinfo(np.int8).max: np.int8, np.iinfo(np.int16).max: np.int16, np.iinfo(np.int32).max: np.int32}
    
    for dtype in ['float64', 'int64']:
        selected_columns = df.select_dtypes(include=[dtype])
        max_values = selected_columns.max()
        types = float_types if dtype == 'float64' else int_types

        for col in selected_columns.columns:
            if df[col].isna().sum() / len(df[col]) > SPARSITY_THRESHOLD:  # If mostly NaN values
                df[col] = pd.arrays.SparseArray(df[col])
            else:
                max_val = max_values[col]
                if not pd.isna(max_val):  # skip columns with only NaNs
                    for max_type_val, type_val in types.items():
                        if max_val <= max_type_val:
                            df[col] = df[col].astype(type_val)
                            break

    for col in df.select_dtypes(include=['object']).columns:
        if df[col].isna().sum() / len(df[col]) > SPARSITY_THRESHOLD:  # If mostly NaN values
            df[col] = pd.arrays.SparseArray(df[col])
        else:
            try:
                df[col] = pd.to_datetime(df[col])
            except ValueError:
                num_unique_values = len(df[col].unique())
                num_total_values = len(df[col])
                if num_unique_values / num_total_values < SPARSITY_THRESHOLD:
                    df[col] = df[col].astype('category')

    return df


### The ``optimize()`` method is fetched from the notebook [memory-usage-optimization-poc.ipynb](https://github.com/harunugurlu/mobile-action-internship/blob/main/Data%20Science/PoC/Memory%20Usage%20Optimization/memory-usage-optimization-poc.ipynb)

In [26]:
data_dir = 'data/rows.csv'

In [27]:
customer_tickets = pd.read_csv(data_dir)

C:\Users\harun\AppData\Local\Temp\ipykernel_12508\291152379.py:1: DtypeWarning: Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  customer_tickets = pd.read_csv(data_dir)


In [37]:
customer_tickets.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,05/10/2019,Checking or savings account,Checking account,Managing an account,Problem using a debit or ATM card,NaN,NaN,NAVY FEDERAL CREDIT UNION,FL,328XX,Older American,NaN,Web,05/10/2019,In progress,Yes,NaN,3238275
1,05/10/2019,Checking or savings account,Other banking product or service,Managing an account,Deposits and withdrawals,NaN,NaN,BOEING EMPLOYEES CREDIT UNION,WA,98204,NaN,NaN,Referral,05/10/2019,Closed with explanation,Yes,NaN,3238228
2,05/10/2019,Debt collection,Payday loan debt,Communication tactics,Frequent or repeated calls,NaN,NaN,CURO Intermediate Holdings,TX,751XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3237964
3,05/10/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Old information reappears or never goes away,NaN,NaN,Ad Astra Recovery Services Inc,LA,708XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3238479
4,05/10/2019,Checking or savings account,Checking account,Managing an account,Banking errors,NaN,NaN,ALLY FINANCIAL INC.,AZ,85205,NaN,NaN,Postal mail,05/10/2019,In progress,Yes,NaN,3238460


Will use the ``Consumer complaint narrative`` to predict the ``Product`` 

In [67]:
print(customer_tickets.shape)

(1282355, 18)


In [63]:
customer_tickets.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1282355 entries, 0 to 1282354
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Date received                 1282355 non-null  object
 1   Product                       1282355 non-null  object
 2   Sub-product                   1047189 non-null  object
 3   Issue                         1282355 non-null  object
 4   Sub-issue                     751169 non-null   object
 5   Consumer complaint narrative  383564 non-null   object
 6   Company public response       449082 non-null   object
 7   Company                       1282355 non-null  object
 8   State                         1262955 non-null  object
 9   ZIP code                      1167057 non-null  object
 10  Tags                          175643 non-null   object
 11  Consumer consent provided?    690654 non-null   object
 12  Submitted via                 1282355 non-

The dataset is very large (1m+ rows, 1.7gb memory usage), let's shrink it down

In [64]:
customer_tickets = optimize(customer_tickets)

In [103]:
customer_tickets.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1282355 entries, 0 to 1282354
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype              
---  ------                        --------------    -----              
 0   Date received                 1282355 non-null  datetime64[ns]     
 1   Product                       1282355 non-null  category           
 2   Sub-product                   1047189 non-null  category           
 3   Issue                         1282355 non-null  category           
 4   Sub-issue                     751169 non-null   category           
 5   Consumer complaint narrative  383564 non-null   Sparse[object, nan]
 6   Company public response       449082 non-null   Sparse[object, nan]
 7   Company                       1282355 non-null  category           
 8   State                         1262955 non-null  category           
 9   ZIP code                      1167057 non-null  category           
 10  Tags  

Inspecting the features of interest (``Consumer complaint narrative`` and ``Product``)

In [68]:
print("Num of non na ticket descriptions", customer_tickets['Consumer complaint narrative'].count())
print("Num of unique products:", customer_tickets['Product'].nunique())

Num of non na ticket descriptions 383564
Num of unique products: 18


We have over __1m__ rows but only less than half of them include a consumer complaint narrative. Lucky for us this number is still large (380k+) and adequate to use in __BERT__. Therefore, we can just drop them. But before that it is a good approach to inspect and to see if there is any correlation between the missing values of  ``Consumer complaint narrative`` and their respective ``Product``.

In [82]:
# Getting the rows with NaN values for 'Consumer complaint narrative'
invalid_tickets = customer_tickets[customer_tickets['Consumer complaint narrative'].isnull()]

In [73]:
invalid_tickets.shape

(898791, 18)

In [92]:
products_complaints = invalid_tickets[['Product', 'Consumer complaint narrative']]

In [99]:
products_complaints['Product'].value_counts()

Mortgage                                                                        225111
Debt collection                                                                 158163
Credit reporting, credit repair services, or other personal consumer reports    133600
Credit reporting                                                                108844
Bank account or service                                                          71321
Credit card                                                                      70352
Student loan                                                                     29875
Checking or savings account                                                      27760
Credit card or prepaid card                                                      26274
Consumer Loan                                                                    22131
Vehicle loan or lease                                                             5632
Money transfer, virtual currency, or money 

Seems like there is no correlation between missing narrative values and product values. There is no large number of certain product. Actually, we should take into account other features such as ``Sub-product``, ``Issue``, ``Sub-issue`` etc. but we are ignoring them for ease.

Get the "valid" tickets with consumer complaint narratives.

In [111]:
valid_tickets = customer_tickets[customer_tickets['Consumer complaint narrative'].notna()].copy()

In [112]:
print(valid_tickets.shape)
valid_tickets.head()

(383564, 18)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
29904,2019-03-23,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account information incorrect,The Summer of XX/XX/2018 I was denied a mortga...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",IL,NaN,NaN,Consent provided,Web,2019-03-23,Closed with explanation,Yes,NaN,3189109
30629,2019-03-22,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,220XX,NaN,Consent provided,Web,2019-03-22,Closed with explanation,Yes,NaN,3187982
30735,2019-03-22,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,770XX,NaN,Consent provided,Web,2019-03-22,Closed with explanation,Yes,NaN,3187954
30795,2019-03-22,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,787XX,NaN,Consent provided,Web,2019-03-22,Closed with explanation,Yes,NaN,3188091
30807,2019-03-22,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,951XX,NaN,Consent provided,Web,2019-03-22,Closed with explanation,Yes,NaN,3188119


In [113]:
valid_tickets.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383564 entries, 29904 to 912553
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype              
---  ------                        --------------   -----              
 0   Date received                 383564 non-null  datetime64[ns]     
 1   Product                       383564 non-null  category           
 2   Sub-product                   331391 non-null  category           
 3   Issue                         383564 non-null  category           
 4   Sub-issue                     269891 non-null  category           
 5   Consumer complaint narrative  383564 non-null  Sparse[object, nan]
 6   Company public response       182680 non-null  Sparse[object, nan]
 7   Company                       383564 non-null  category           
 8   State                         382178 non-null  category           
 9   ZIP code                      294787 non-null  category           
 10  Tags            

We have converted the dtype of the ``Consumer complaint narrative`` to Sparse array because it had lots of missing values. But in our ``valid_tickets`` DataFrame, we no longer have missing values of ``Consumer complaint narrative``. Therefore, we should cast it back to dense array.

In [114]:
valid_tickets.loc[:, 'Consumer complaint narrative'] = valid_tickets['Consumer complaint narrative'].sparse.to_dense()

Should check the other Sparse array columns as well.

In [115]:
valid_tickets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383564 entries, 29904 to 912553
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype              
---  ------                        --------------   -----              
 0   Date received                 383564 non-null  datetime64[ns]     
 1   Product                       383564 non-null  category           
 2   Sub-product                   331391 non-null  category           
 3   Issue                         383564 non-null  category           
 4   Sub-issue                     269891 non-null  category           
 5   Consumer complaint narrative  383564 non-null  object             
 6   Company public response       182680 non-null  Sparse[object, nan]
 7   Company                       383564 non-null  category           
 8   State                         382178 non-null  category           
 9   ZIP code                      294787 non-null  category           
 10  Tags            

In [22]:
# We have 13 unique values, the min is 0 and the max is 12 which means we have all the values between 0-12.

print("row count: ", customer_tickets.shape[0])
print("Ticket category unique values: ", customer_tickets['category'].nunique())
print("Min value of category:", customer_tickets['category'].min())
print("Min value of category:", customer_tickets['category'].max())

Ticket category unique values:  13
row count:  48549
Min value of category: 0
Min value of category: 12
True


In [17]:
print("Num of missing values in \"category\" column:", customer_tickets['category'].isnull().sum())
print("Num of missing values in \"title\" column:", customer_tickets['title'].isnull().sum())
print("Num of missing values in \"body\" column:", customer_tickets['title'].isnull().sum())

Num of missing values in "category" column: 0
Num of missing values in "title" column: 712
Num of missing values in "body" column: 712


Since there are 712 missing values in the body column, we need to clean it. But first, let's see if the value 0 in the ``category`` column indicates missing value. If we have zeroes in the ``category`` where the ``body`` is missing, it might be an indicator that 0 is not a valid category.

In [25]:
print("The value counts in the \"category\" column:", customer_tickets['category'].value_counts())

The value counts in the "category" column: 4     34061
5      9634
6      2628
7       921
11      612
8       239
9       191
3       137
1        72
12       45
0         4
2         3
10        2
Name: category, dtype: int64


Let's check the what categories do the tickets with missing bodies fall in. In order to see if there is a correlation between them. For example if 650 of the 712 tickets with missing bodies has the same category, it might indicate that tickets with missing bodies have a valid category.

In [ ]:
tickets_missing_bodies = for body in customer_tickets['body']

No, there doesn't seem to be a relationship between the missing ``body`` values and zeroes in ``category``. Perhaps, '0' is a valid category for a ticket.

Finally, we can start training BERT

#### Bert expects the data in a certain form:
- __Column 0:__ An ID for the row
- __Column 1:__ The label for the row (should be an int — class labels: 0,1,2,3 etc) This column represents the class or category of the ticket (the "target" that the model is trying to predict).
- __Column 2:__ A column of the same letter for all rows — this is a throw-away column that we need to include because BERT expects it.  It doesn't have any impact on the task.
- __Column 3:__ The text examples we want to classify. This is the data that BERT will use to learn how to classify tickets.

Also it expects the data to be in tsv file.

In [118]:
indices = range(valid_tickets['Consumer complaint narrative'].size)

In [127]:
# Getting product and complaint pairs
product_complaint_pairs = valid_tickets[['Product', 'Consumer complaint narrative']]

In [135]:
product_complaint_pairs.head()

,Product,Consumer complaint narrative
29904,"Credit reporting, credit repair services, or o...",The Summer of XX/XX/2018 I was denied a mortga...
30629,"Credit reporting, credit repair services, or o...",There are many mistakes appear in my report wi...
30735,"Credit reporting, credit repair services, or o...",There are many mistakes appear in my report wi...
30795,"Credit reporting, credit repair services, or o...",There are many mistakes appear in my report wi...
30807,"Credit reporting, credit repair services, or o...",There are many mistakes appear in my report wi...


In [147]:
unique_products = pd.DataFrame(valid_tickets['Product'].unique())

In [149]:
unique_products.index

RangeIndex(start=0, stop=18, step=1)

Mapping the Products to integers.

In [150]:
from sklearn.preprocessing import LabelEncoder

# Initializing a LabelEncoder
le = LabelEncoder()

# Fitting the encoder to the 'Product' data
valid_tickets['Product_encoded'] = le.fit_transform(valid_tickets['Product'])

In [158]:
data = {'ID': range(len(valid_tickets)), 'Product': valid_tickets['Product_encoded'], 'Throw away': 'a', 'Complaint': valid_tickets['Consumer complaint narrative']}

In [159]:
df_final = pd.DataFrame(data=data)

In [161]:
df_final.head()

,ID,Product,Throw away,Complaint
29904,0,6,a,The Summer of XX/XX/2018 I was denied a mortga...
30629,1,6,a,There are many mistakes appear in my report wi...
30735,2,6,a,There are many mistakes appear in my report wi...
30795,3,6,a,There are many mistakes appear in my report wi...
30807,4,6,a,There are many mistakes appear in my report wi...
